# ДЗ 1

### 1) Сколько твитов в наборе?

Создаем список из твитов, считаем его длину

In [135]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [137]:
import json
from collections import Counter
from pprint import pprint

twitter = []
with open('hw_3_twitter.json', encoding='utf-8') as f:
    for line in f:
        twitter.append(json.loads(line))
print('В наборе ', len(twitter), ' твитов')

В наборе  2556  твитов


### 2) Какой процент твитов составляют удаленные записи? (помеченные как deleted)

Проверяем каждый твит на deleted и создаем новый словарь неудаленных твитов

In [138]:
nd_twitter = []
for tweet in twitter:
    if tweet.get('delete') is None:
        nd_twitter.append(tweet)
print(100 - 100*len(nd_twitter)/len(twitter), ' процентов')

14.162754303599371  процентов


### 3) Какие самые популярные языки твитов?
создаем каунтер для языков, заполняем его, смотрим

In [139]:
langs = Counter()
for tweet in nd_twitter:
    langs[tweet.get('lang')] += 1
for item in langs.most_common(10):
    print(item[0], end=', ')

en, ja, es, ko, th, ar, und, in, pt, fr, 

### 4) Есть ли твиты от одного и того же пользователя? Если да, то сколько таких пользователей?
Делаем каунтер для айди, заполняем его
Проверяем есть ли в нем значения больше 1 и считаем сколько их

In [140]:
ids = Counter()
for tweet in nd_twitter:
    ids[tweet.get('user').get('id')] += 1
many_tweets = 0
for item in ids:
    if ids[item] > 1:
        many_tweets += 1
if many_tweets > 0:
    print('такие пользователи есть')
print('их ', many_tweets)

такие пользователи есть
их  25


### 5) Топ-20 хэштегов
создаем каунтер хэштегов, заполняем его, выводим самые популярные

In [141]:
hts = Counter()
for tweet in nd_twitter:
    if tweet.get('entities') is not None:
        if tweet.get('entities').get('hashtags') is not None:
            for ht in tweet.get('entities').get('hashtags'):
                hts[ht.get('text')] += 1
for item in hts.most_common(20):
    print(item[0], end=', ')

BTS, 방탄소년단, AMAs, 人気投票ガチャ, 태형, 뷔, BTSinChicago, BTSLoveYourselfTour, 오늘의방탄, PledgeForSwachhBharat, MPN, PCAs, V, 시카고1회차공연, เป๊กผลิตโชค, JIMIN, running, NCT, 지민, WajahmuPlastik, 

### 6) Предобработать тексты оригинальных твитов (не ретвитов) на английском языке (убрать пунктуацию, привести к нижнему регистру) и составить частотный словарь

Сначала идут две вспомогательные функции для очищения

Потом делается список оригинальных твитов на английском языке

Потом делается список слов из этих твитов

Потом создается частотный словарь

In [149]:
def clear_text(text):
    trash_symbols = '!"#$%&\'-()*+,./:;<=>?@[\\]^_`{|}~«»—'
    return text.strip(trash_symbols)


def get_words(string_of_text):
    good_words = []
    for word in string_of_text.split():
        candidate = clear_text(word)
        if candidate != '':
            candidate = candidate.lower()
            good_words.append(candidate)
    return good_words


or_tweets = []
for tweet in nd_twitter:
    en = (tweet.get('lang') == 'en')
    rs = (tweet.get('retweeted_status') is None)
    qs = (tweet.get('quoted_status') is None)
    if rs and qs and en:
        or_tweets.append(tweet)

words = []
for tweet in or_tweets:
    words.extend(get_words(tweet.get('text')))

wordc = Counter()
for word in words:
    wordc[word] += 1
print(wordc.most_common(10))

[('the', 106), ('to', 79), ('a', 65), ('i', 60), ('and', 57), ('you', 45), ('for', 40), ('is', 40), ('of', 40), ('it', 37)]


### 7) Найти количество подписчиков (фолловеров) у авторов твитов и вывести топ-10

создаем словарь, где ключи -- айди пользователя, значения -- кол-во подписчиков  
выводим самых популярных

In [146]:
fol_dict = Counter()
for tweet in nd_twitter:
    name = tweet.get('user').get('id')
    followers = tweet.get('user').get('followers_count')
    fol_dict[name] = followers
print('количество подписчиков у самых популярных авторов:')
for i in fol_dict.most_common(10):
    print(i[1], end=', ')

количество подписчиков у самых популярных авторов:
2521403, 1491309, 1206759, 1137374, 625463, 392472, 383698, 374222, 318189, 311319, 

### 8) Топ-10 источников твита (из какого приложения написан)  
создаем каунтер для источников твита, которые вырезаем с помощую регулярок, выводим самые популярные

In [147]:
import re
sources = Counter()
for tweet in nd_twitter:
    src = tweet.get('source')
    name = re.search('>(.*?)<', src).group(1)
    sources[name] += 1
for i in sources.most_common(10):
    print(i[0], end=', ')

Twitter for iPhone, Twitter for Android, Twitter Web Client, twittbot.net, Twitter Lite, Twitter for iPad, TweetDeck, Facebook, IFTTT, تطبيق قرآني, 